# Paper Figures
1. Map of regions
2. `NPP` predictability heatmap
3. `SIE` predictability heatmap
4. "Propagation" of predictability (`SIE`, `MLD`, `SFC_IRR`, `NPP`, and maybe `SFC_CHL`, `SFC_BIOMASS`) 
5. `SIE` predicting `NPP` cross correlation
6. `SFC_IRR` predicting `NPP` cross correlation

In [1]:
import xarray as xr
import numpy as np
import scipy.stats as stats
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean.cm as cmo

In [2]:
mpl.rcParams['figure.dpi'] = 500
# %config InlineBackend.figure_formats = ['png']
%config InlineBackend.figure_formats = ['pdf']

In [3]:
def open_metric(var, reg, metric, timescale='monthly', ens_type='', ctrl_type=''):
    
    writedir = '/local/projects/so_predict/bbuchovecky/storage/so_predict_'+ctrl_type+'derived/'
    
    if metric.lower() == 'clim':
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+metric+'.nc'
    
    if metric.lower() == 'anom' or metric.lower() == 'mean' or (metric.lower() == 'var' and timescale.lower() == 'monthly'):
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+metric+'.nc'
    
    if metric.lower() == 'ppp':
        subdir = 'PPP/'+var.upper()+'/'
        if ens_type != '':
            ens_type += '_'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+ens_type+'ppp.nc'
        
    return xr.open_dataset(writedir+subdir+filename)

def get_plotting_labels():
    with open('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb') as handle:
        plotting_dicts = pkl.load(handle)
    
    reg_names = plotting_dicts['reg_names_A/B']
    var_abbrv_names = plotting_dicts['var_abbrv_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']
    month_letters = plotting_dicts['month_letters']
    var_su_names = plotting_dicts['var_su_names']
    
    return reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names

## Predictability Heatmap 

In [4]:
def ppp_simple_heatmap(
    var, vmin=0, threshold=0.183, hatch='//', cmap=cmo.matter, figsize=(10,6), titlefontsize='large', labelfontsize='large'):

    file = open('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
    plotting_dicts = pkl.load(file)
    file.close()

    reg_names = plotting_dicts['reg_names_A/B']
    var_su_names = plotting_dicts['var_su_names']
    var_lu_names = plotting_dicts['var_lu_names']
    var_ll_names = plotting_dicts['var_ll_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']

    ppp = open_metric(var, 'so', 'ppp')
    regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

    fig,ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=figsize)
    
    subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

    for (ireg,reg) in zip(range(len(regions)), regions):
        ppp_matrix = np.zeros((10,12))

        for m in range(ppp['nT'].size):
            ppp_matrix[int(m/12),m%12] = ppp[reg][m]

        ppp_matrix = ppp_matrix.T

        im = ax[int(ireg/3),ireg%3].pcolormesh(ppp_matrix, vmin=vmin, vmax=1.0, cmap=cmap, shading='auto')

        ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', fontsize=titlefontsize, loc='left')

        ax[int(ireg/3),ireg%3].set_yticks(np.arange(ppp_matrix.shape[0]) + 0.5)
        ax[int(ireg/3),ireg%3].set_yticklabels(abbrv_month_names, fontsize=labelfontsize)

        ax[int(ireg/3),ireg%3].set_xticks(np.arange(ppp_matrix.shape[1]) + 0.5)
        ax[int(ireg/3),ireg%3].set_xticklabels(np.arange(1,11,1), fontsize=labelfontsize)
        ax[int(ireg/3),ireg%3].set_aspect('auto')

        if int(ireg/3) == 1:
            ax[int(ireg/3),ireg%3].set_xlabel('Year', fontsize=labelfontsize)

        if hatch:
            masked = np.ma.masked_array(ppp_matrix, mask=np.where(ppp_matrix > threshold, 0, 1))
            masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)

    plt.suptitle('PM predictability of '+var_su_names[var], x=0.424, fontweight='bold', fontsize=titlefontsize+4)
    fig.tight_layout()

    cb = fig.colorbar(im, ax=ax.ravel().tolist(), pad=0.06)
    cb.ax.fill_between([-0.1,1.1], [1.0,1.0], [threshold,threshold], hatch=hatch, alpha=0)
    cb.ax.plot([-0.1,1.1], [threshold,threshold], color='k', linewidth=1)
    cb.ax.set_ylabel('PPP', fontsize=labelfontsize)

    cb.ax.arrow(x=-0.5, y=0+0.01, dx=0, dy=0.183-0.01, clip_on=False, length_includes_head=True, 
                width=0.1, head_length=0.02, head_width=0.4,
                facecolor='grey', edgecolor='grey')
    cb.ax.arrow(x=-0.5, y=0.183-0.01, dx=0, dy=-0.183+0.01, clip_on=False, length_includes_head=True, 
                width=0.1, head_length=0.02, head_width=0.4,
                facecolor='grey', edgecolor='grey')
    cb.ax.text(x=-1, y=0.183 + (1-0.183)/2, s='Predictable', clip_on=False, ha='center', va='center', rotation='vertical',
              color='grey', fontsize=labelfontsize-1, fontweight='bold')
    
    cb.ax.arrow(x=-0.5, y=1-0.01, dx=0, dy=0.183-1+0.01, clip_on=False, length_includes_head=True, 
                width=0.1, head_length=0.02, head_width=0.4,
                facecolor='grey', edgecolor='grey')
    cb.ax.arrow(x=-0.5, y=0.183+0.01, dx=0, dy=1-0.183-0.01, clip_on=False, length_includes_head=True, 
                width=0.1, head_length=0.02, head_width=0.4,
                facecolor='grey', edgecolor='grey')
    cb.ax.text(x=-1, y=0.183/2, s='Not predictable', clip_on=False, ha='center', va='center', rotation='vertical',
              color='grey', fontsize=labelfontsize-1, fontweight='bold')

In [5]:
ppp_simple_heatmap('npp', threshold=0.183, labelfontsize=10, titlefontsize=12)

<Figure size 5000x3000 with 7 Axes>

In [6]:
ppp_simple_heatmap('sie', threshold=0.183, labelfontsize=12, titlefontsize=14)

<Figure size 5000x3000 with 7 Axes>

## Predictability Timeseries

In [7]:
file = open('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
plotting_dicts = pkl.load(file)
file.close()

reg_names = plotting_dicts['reg_names_A/B']
var_su_names = plotting_dicts['var_su_names']
var_lu_names = plotting_dicts['var_lu_names']
var_ll_names = plotting_dicts['var_ll_names']
var_abbrv_names = plotting_dicts['var_abbrv_names']
abbrv_month_names = plotting_dicts['abbrv_month_names']
var_colors = plotting_dicts['var_colors']
month_letters = plotting_dicts['month_letters']

In [8]:
def ppp_timeseries(
    variables=['SIE','MLD','SFC_IRR','NPP'], threshold=0.183, endmonth=14, ylim=(-0.2,1),
    figsize=(10,6), titlefontsize='large', labelfontsize='large'):

    file = open('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
    plotting_dicts = pkl.load(file)
    file.close()

    reg_names = plotting_dicts['reg_names_A/B']
    var_su_names = plotting_dicts['var_su_names']
    var_lu_names = plotting_dicts['var_lu_names']
    var_ll_names = plotting_dicts['var_ll_names']
    var_abbrv_names = plotting_dicts['var_abbrv_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']
    var_colors = plotting_dicts['var_colors']
    month_letters = plotting_dicts['month_letters']
    
    var_abbrv_names['sfc_fed']='$\mathrm{FED_{SFC}}$'
    var_abbrv_names['sfc_irr']='$\mathrm{IRR_{SFC}}$'
    
    regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

    fig,ax = plt.subplots(2, 3, figsize=figsize)
    subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

    for var in variables:
        ppp = open_metric(var, 'so', 'ppp')
        for (ireg,reg) in enumerate(regions):
            
            if var.lower() == 'npp':
                ax[int(ireg/3),ireg%3].plot(ppp[reg].values[:endmonth], color=var_colors[var.lower()],
                                            label=var_abbrv_names[var.lower()], linewidth=1)
            if var.lower() != 'npp':
                ax[int(ireg/3),ireg%3].plot(ppp[reg].values[:endmonth], color=var_colors[var.lower()],
                                            label=var_abbrv_names[var.lower()], linewidth=0.5)
            
            ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', fontsize=titlefontsize, loc='left')
            
            if var == variables[-1]:
                ax[int(ireg/3),ireg%3].axhline(threshold, 0, endmonth, color='grey', ls='--',
                                               label='Threshold for\npredictability\n(PPP=0.183)',
                                               linewidth=0.5)
            if var != variables[-1]:
                ax[int(ireg/3),ireg%3].axhline(threshold, 0, endmonth, color='grey', ls='--',
                                               linewidth=0.5)

            ax[int(ireg/3),ireg%3].axvline(10, ylim[0], ylim[1], color='grey', ls=':', alpha=0.5, linewidth=0.5)    

            ax[int(ireg/3),ireg%3].set_ylim(ylim[0],ylim[1])
            ax[int(ireg/3),ireg%3].set_xlim(0,endmonth-1)
            ax[int(ireg/3),ireg%3].set_xticks(np.arange(0,endmonth,2))
            ax[int(ireg/3),ireg%3].set_xticklabels(np.array(2*abbrv_month_names)[np.arange(0,endmonth,2)])
            ax[int(ireg/3),ireg%3].tick_params(axis='both', which='major', labelsize=labelfontsize-2)

            if int(ireg/3) == 1:
                ax[int(ireg/3),ireg%3].set_xlabel('Forecast month', fontsize=labelfontsize)
                
            if ireg%3 == 0:
                ax[int(ireg/3),ireg%3].set_ylabel('PPP', fontsize=labelfontsize)
             
            if ireg == 2:
                ax[int(ireg/3),ireg%3].legend(bbox_to_anchor=(1.02, 1.02), loc='upper left', fontsize=labelfontsize-2)
                
            if ireg == len(regions)-1:
                ax[int(ireg/3),ireg%3].arrow(x=endmonth-1+0.75, dx=0, y=ylim[0]+0.01, dy=-ylim[0]+0.183-0.01, clip_on=False, 
                                             length_includes_head=True, width=0.1, head_length=0.05, head_width=0.4, 
                                             facecolor='grey', edgecolor='grey')
                ax[int(ireg/3),ireg%3].arrow(x=endmonth-1+0.75, dx=0, y=0.183-0.01, dy=ylim[0]-0.182+0.01, clip_on=False, 
                                             length_includes_head=True, width=0.1, head_length=0.05, head_width=0.4, 
                                             facecolor='grey', edgecolor='grey')
                ax[int(ireg/3),ireg%3].text(x=endmonth+0.5, y=0.183 + (ylim[1]-0.183)/2, s='Predictable', clip_on=False, 
                                            ha='center', va='center', rotation='vertical', color='grey',
                                            fontsize=labelfontsize-2, fontweight='normal')

                ax[int(ireg/3),ireg%3].arrow(x=endmonth-1+0.75, dx=0, y=ylim[1]-0.01, dy=0.183-ylim[1]+0.01, clip_on=False, 
                                             length_includes_head=True, width=0.1, head_length=0.05, head_width=0.4, 
                                             facecolor='grey', edgecolor='grey')
                ax[int(ireg/3),ireg%3].arrow(x=endmonth-1+0.75, dx=0, y=0.183+0.01, dy=ylim[1]-0.183-0.01, clip_on=False, 
                                             length_includes_head=True, width=0.1, head_length=0.05, head_width=0.4, 
                                             facecolor='grey', edgecolor='grey')
                ax[int(ireg/3),ireg%3].text(x=endmonth+0.75, y=ylim[0] - (ylim[0]-0.183)/2, s='Not\npredictable', clip_on=False, 
                                            ha='center', va='center', rotation='vertical', color='grey',
                                            fontsize=labelfontsize-2, fontweight='normal')
            
        
    fig.suptitle('Time series of first year PM predictability', x=0.47, ha='center', fontweight='bold', fontsize=titlefontsize+2)
    fig.tight_layout()

In [9]:
ppp_timeseries(variables=['SIE','SST','MLD','SFC_IRR','SFC_FED','NPP'], labelfontsize=10, titlefontsize=12, figsize=(10,6))

<Figure size 5000x3000 with 6 Axes>

In [11]:
ppp_timeseries(variables=['SIE','SST','SFC_IRR','NPP'], labelfontsize=10, titlefontsize=12, figsize=(10,6))

<Figure size 5000x3000 with 6 Axes>

## Progression of Predictability

In [49]:
def ppp_var_heatmap(
    variables, reg, num_months=24, vmin=-0.2, threshold=0.183, hatch='\\', yaxis_steps=2, cmap=cmo.matter, figsize=(8,7), titlefontsize=16,
    labelfontsize=14):
    
    file = open('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
    plotting_dicts = pkl.load(file)
    file.close()

    reg_names = plotting_dicts['reg_names_A/B']
    var_su_names = plotting_dicts['var_su_names']
    var_su_names = {'npp': 'NPP',
                     'mld': 'MLD',
                     'sie': 'SIE',
                     'sst': 'SST',
                     'sss': 'SSS',
                     'cn_inv': 'SIC',
                     'pco2surf': 'Surface pCO$_2$',
                     'siv': 'SIV',
                     'sfc_fed': '$\mathrm{FED_{SFC}}$',
                     'sfc_irr': '$\mathrm{IRR_{SFC}}$',
                     'sfc_biomass': 'BIOM',
                     'sfc_chl': 'CHL'}
    abbrv_month_names = plotting_dicts['abbrv_month_names']

    regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

    variable_names = []
    
    if reg.lower() == 'all':
        
        fig,ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=figsize)
        
        subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

        for (ireg,reg) in zip(range(len(regions)), regions):
            ppp_matrix = np.zeros((len(variables),num_months))

            for (i,var) in zip(range(len(variables)), variables):
                ppp = open_metric(var, 'so', 'ppp')
                ppp_matrix[i] = ppp[reg][0:num_months]
                if ireg == 0:
                    variable_names.append(var_su_names[var])

            ppp_matrix = ppp_matrix.T

            im = ax[int(ireg/3),ireg%3].pcolormesh(ppp_matrix, vmin=vmin, vmax=1.0, cmap=cmap, shading='auto')

#             ax[int(ireg/3),ireg%3].set_yticks(np.arange(0,ppp_matrix.shape[0],yaxis_steps) + 0.5)            
#             ax[int(ireg/3),ireg%3].set_yticklabels(np.arange(0,ppp_matrix.shape[0],yaxis_steps), fontsize=labelfontsize)
            
            yticks = np.arange(0,ppp_matrix.shape[0],yaxis_steps)
            yticklabels = [abbrv_month_names[i] for i in yticks%12]
            ax[int(ireg/3),ireg%3].set_yticks(yticks + 0.5)            
            ax[int(ireg/3),ireg%3].set_yticklabels(yticklabels, fontsize=labelfontsize)
            
            if ireg%3 == 0:
                ax[int(ireg/3),ireg%3].set_ylabel('Forecast month', fontsize=labelfontsize+1)

            ax[int(ireg/3),ireg%3].set_xticks(np.arange(ppp_matrix.shape[1]) + 0.5)
            ax[int(ireg/3),ireg%3].set_xticklabels(variable_names, fontsize=labelfontsize)
            plt.setp(ax[int(ireg/3),ireg%3].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=labelfontsize)
            
            ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)
    
            if hatch:
                masked = np.ma.masked_array(ppp_matrix, mask=np.where(ppp_matrix > threshold, 0, 1))
                masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)
#                 masked = np.ma.masked_array(ppp_matrix, mask=np.where((ppp_matrix > 0.183) & (ppp_matrix < threshold), 0, 1))
#                 masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch='//', alpha=0)

#         fig.suptitle('1$^{st}$ Year PM Predictability', x=0.424, fontweight='bold', fontsize=titlefontsize+4)
#         fig.suptitle('PM predictability of spring bloom drivers', x=0.424, fontweight='bold', fontsize=titlefontsize+4) # location for figsize=(13,10) or (10,8)
        fig.suptitle('PM predictability of spring bloom drivers', x=0.435, fontweight='bold', fontsize=titlefontsize+2) # location for figsize=(13,10) or (10,8)
        fig.tight_layout()

        cb = fig.colorbar(im, ax=ax.ravel().tolist(), pad=0.07)
        cb.ax.plot([-0.1,1.1], [threshold,threshold], color='k', linewidth=1)
        cb.ax.plot([-0.1,1.1], [0.183,0.183], color='k', linewidth=1)
        cb.ax.fill_between([-0.1,1.1], [1.0,1.0], [threshold,threshold], hatch=hatch, alpha=0)
        cb.set_label('PPP', fontsize=labelfontsize+1)
        for t in cb.ax.get_yticklabels():
            t.set_fontsize(labelfontsize)
#         cb.ax.fill_between([-0.1,1.1], [1.0,1.0], [0.183,0.183], hatch='//', alpha=0)
        
        cb.ax.arrow(x=-0.5, y=0+0.01, dx=0, dy=0.183-0.01, clip_on=False, length_includes_head=True, 
                width=0.1, head_length=0.02, head_width=0.4,
                facecolor='grey', edgecolor='grey')
        cb.ax.arrow(x=-0.5, y=0.183-0.01, dx=0, dy=-0.183+0.01, clip_on=False, length_includes_head=True, 
                    width=0.1, head_length=0.02, head_width=0.4,
                    facecolor='grey', edgecolor='grey')
        cb.ax.text(x=-1.5, y=0.183 + (1-0.183)/2, s='Predictable', clip_on=False, ha='center', va='center', rotation='vertical',
                  color='grey', fontsize=labelfontsize, fontweight='bold')

        cb.ax.arrow(x=-0.5, y=1-0.01, dx=0, dy=0.183-1+0.01, clip_on=False, length_includes_head=True, 
                    width=0.1, head_length=0.02, head_width=0.4,
                    facecolor='grey', edgecolor='grey')
        cb.ax.arrow(x=-0.5, y=0.183+0.01, dx=0, dy=1-0.183-0.01, clip_on=False, length_includes_head=True, 
                    width=0.1, head_length=0.02, head_width=0.4,
                    facecolor='grey', edgecolor='grey')
        cb.ax.text(x=-1.5, y=0.183/2, s='Not predictable', clip_on=False, ha='center', va='center', rotation='vertical',
                  color='grey', fontsize=labelfontsize, fontweight='bold')

#         cb.ax.text(0.5, vmin-0.03, 'No skill', ha='center', va='center', fontsize=labelfontsize)
#         cb.ax.text(0.5, 1.0+0.03, 'Perfect skill', ha='center', va='center', fontsize=labelfontsize)

In [51]:
variables = ['sie', 'mld', 'sfc_irr', 'npp']
ppp_var_heatmap(variables, 'all', num_months=14, figsize=(10,8), vmin=0, threshold=0.25, labelfontsize=12, titlefontsize=12, hatch='xx');

<Figure size 5000x4000 with 7 Axes>

In [52]:
variables = ['sie', 'mld', 'sfc_irr', 'npp', 'sfc_chl', 'sfc_biomass']
ppp_var_heatmap(variables, 'all', num_months=14, figsize=(13,10), vmin=0, threshold=0.183, hatch='//');

<Figure size 6500x5000 with 7 Axes>

In [15]:
mpl.rcParams['hatch.linewidth'] = 0.5
variables = ['sie', 'mld', 'sfc_irr', 'npp', 'sfc_chl', 'sfc_biomass']
ppp_var_heatmap(variables, 'all', num_months=14, figsize=(10,8), vmin=0, threshold=0.183, hatch='///', titlefontsize=12, labelfontsize=10);

<Figure size 5000x4000 with 7 Axes>

In [17]:
mpl.rcParams['hatch.linewidth'] = 0.5
variables = ['mld', 'sfc_fed', 'npp', 'sfc_chl', 'sfc_biomass']
ppp_var_heatmap(variables, 'all', num_months=14, figsize=(10,8), vmin=0, threshold=0.183, hatch='///', titlefontsize=12, labelfontsize=10);

<Figure size 5000x4000 with 7 Axes>

In [16]:
mpl.rcParams['hatch.linewidth'] = 0.25
variables = ['sie', 'mld', 'sfc_irr', 'npp', 'sfc_chl', 'sfc_biomass']
ppp_var_heatmap(variables, 'all', num_months=14, figsize=(5.5,4), vmin=0, threshold=0.25, hatch='xxxx', titlefontsize=6, labelfontsize=5);

<Figure size 2750x2000 with 7 Axes>

## Cross Correlation

In [54]:
## predictor --> the field doing the predicting (e.g., SIE or SFC_IRR)
## predictand --> the field being predicted (e.g., NPP)
def compute_lagged_xcorr(predictor, predictand, maxlag):
    ## convert DataArray to NumPy array
    predictor = predictor.values
    predictand = predictand.values
    
    assert predictor.size == predictand.size, 'predictor and predictand have different lengths'
    N = predictor.size
    
    ## rows are different init months, cols are different lags
    r_matrix = np.zeros((abs(maxlag)+1, 12))
    p_matrix = np.zeros((abs(maxlag)+1, 12))

    init_matrix = np.zeros((abs(maxlag)+1, 12))
    lag_matrix = np.zeros((abs(maxlag)+1, 12))
    
    lag_values = np.arange(maxlag, 1)
    for (it,init) in enumerate(range(0,12)):
        for (ig,lag) in enumerate(lag_values):   
            trim = 12*((abs(lag)-1)//12+1)
            init_matrix[ig][it] = init
            lag_matrix[ig][it] = lag
            
            if lag == 0:
                tmp_predictand = predictand[init:N:12]
                tmp_predictor = predictor[init:N:12]
                
            else:
                tmp_predictand = predictand[init+trim:N:12]
                tmp_predictor = predictor[init+trim+lag:N-trim+init:12]
            
            r_matrix[ig][it], p_matrix[ig][it] = stats.pearsonr(tmp_predictand, tmp_predictor)
            
    return r_matrix, p_matrix, init_matrix, lag_matrix

##############

def plot_regional_lagged_xcorr(predictor, predictand, maxlag=-13, threshold=0.30, threshold_arr=False, hatch='...', figsize=(12,8), labelfontsize=12, titlefontsize=14):
    plt.rcParams.update({'mathtext.default':'regular'})
    reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names = get_plotting_labels()
    region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']

    fig,ax = plt.subplots(2, 3, figsize=figsize)
    
    subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
    
    for (ireg,reg) in enumerate(region_list):  
        ts_predictor = open_metric(predictor, 'so', 'anom')[reg]
        ts_predictand = open_metric(predictand, 'so', 'anom')[reg]

        r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)
    
        ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
        numlags = r.shape[0]
        r_transformed = np.empty(shape=r.shape)
        p_transformed = np.empty(shape=p.shape)
        for i in range(numlags):
            r_transformed[numlags-i-1] = r[i]
            p_transformed[numlags-i-1] = p[i]
        r = r_transformed
        p = p_transformed
        
        im = ax[int(ireg/3),ireg%3].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)
        
        if hatch:
            if threshold_arr:
                masked = np.ma.masked_array(r, mask=np.where(abs(r) > threshold[ireg], 0, 1))
                masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)
            else:
                masked = np.ma.masked_array(r, mask=np.where(abs(r) > threshold, 0, 1))
                masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)
        
        ax[int(ireg/3),ireg%3].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
        ax[int(ireg/3),ireg%3].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
        ax[int(ireg/3),ireg%3].set_xticks((np.arange(r.shape[1]) + 0.5))
        ax[int(ireg/3),ireg%3].set_xticklabels(month_letters, fontsize=labelfontsize)
        ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)
        
        if ireg%3 == 0:
            ax[int(ireg/3),ireg%3].set_ylabel('Lead (month)', fontsize=labelfontsize)   

    fig.suptitle(r'correlation ( '+var_su_names[predictand]+'$_{target\:month}$, '+var_su_names[predictor]+'$_{lead\:month}$ )', ha='center', x=0.45, fontweight='bold', fontsize=titlefontsize+4)
    fig.tight_layout()
    
    cb = fig.colorbar(im, ax=ax.ravel().tolist())
    cb.set_label('Correlation coefficient', fontsize=labelfontsize)
    
#     if hatch:
#         cb.ax.plot([-1,1], [threshold,threshold], color='k')
#         cb.ax.plot([-1,1], [-threshold,-threshold], color='k')
#         cb.ax.fill_between([-1,1], [1,1], [threshold,threshold], hatch=hatch, alpha=0)
#         cb.ax.fill_between([-1,1], [-threshold,-threshold], [-1,-1], hatch=hatch, alpha=0)

### SIE predicting NPP

In [55]:
predictor = 'sie'
predictand = 'npp'
sie_pred_npp_rthresh = np.array([0.28028028, 0.15115115, 0.18318318, 0.24524525, 0.15115115, 0.14614615])
plot_regional_lagged_xcorr(predictor, predictand, threshold=sie_pred_npp_rthresh, threshold_arr=True, figsize=(10,7), labelfontsize=12, titlefontsize=12);

<Figure size 5000x3500 with 7 Axes>

### SFC_IRR predicting NPP

In [57]:
predictor = 'sfc_irr'
predictand = 'npp'
irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
plot_regional_lagged_xcorr(predictor, predictand, threshold=irr_pred_npp_rthresh, threshold_arr=True, figsize=(10,7), labelfontsize=12, titlefontsize=12);

<Figure size 5000x3500 with 7 Axes>

### SFC_FED predicting NPP

In [58]:
predictor = 'sfc_fed'
predictand = 'npp'
# irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
plot_regional_lagged_xcorr(predictor, predictand, threshold=0.2, threshold_arr=False, figsize=(10,7), labelfontsize=12, titlefontsize=12);

<Figure size 5000x3500 with 7 Axes>

### SST predicting NPP

In [60]:
predictor = 'sst'
predictand = 'npp'
# irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
plot_regional_lagged_xcorr(predictor, predictand, threshold=0.2, threshold_arr=False, figsize=(10,7), labelfontsize=12, titlefontsize=12);

<Figure size 5000x3500 with 7 Axes>

### MLD predicting NPP

In [59]:
predictor = 'mld'
predictand = 'npp'
# irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
plot_regional_lagged_xcorr(predictor, predictand, threshold=0.2, threshold_arr=False, figsize=(10,7), labelfontsize=12, titlefontsize=12);

<Figure size 5000x3500 with 7 Axes>

### Final main xcorr figure

In [22]:
np.set_printoptions(precision=2, linewidth=150)

In [25]:
# predictor
# predictand
maxlag=-13
sie_pred_npp_rthresh = np.array([0.28028028, 0.15115115, 0.18318318, 0.24524525, 0.15115115, 0.14614615])
irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
hatch='...'
figsize=(13,6)
labelfontsize=10
titlefontsize=12


plt.rcParams.update({'mathtext.default':'regular'})

reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names = get_plotting_labels()
region_list = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

fig,ax = plt.subplots(2, 6, figsize=figsize, constrained_layout=True)

subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('SIE', 'so', 'anom')[reg]
    ts_predictand = open_metric('NPP', 'so', 'anom')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed
    
    print('SIE')
    print(reg)
    print(r.shape)
    print(r)

    im = ax[0,ireg].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > sie_pred_npp_rthresh[ireg], 0, 1))
    masked_im = ax[0,ireg].pcolor(masked, hatch=hatch, alpha=0)

    ax[0,ireg].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[0,ireg].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[0,ireg].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[0,ireg].set_xticklabels(month_letters, fontsize=labelfontsize)
    ax[0,ireg].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)

    if ireg == 0:
        ax[0,ireg].set_ylabel('Lead (month)', fontsize=labelfontsize)
        ax[0,ireg].text(-5.3, -maxlag/2+0.75, 'correlation (NPP, SIE)', clip_on=False, rotation='vertical', ha='center', va='center', fontsize=titlefontsize, fontweight='bold')

for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('SFC_IRR', 'so', 'anom')[reg]
    ts_predictand = open_metric('NPP', 'so', 'anom')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed
    
    print('IRR')
    print(reg)
    print(r)

    im = ax[1,ireg].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > irr_pred_npp_rthresh[ireg], 0, 1))
    masked_im = ax[1,ireg].pcolor(masked, hatch=hatch, alpha=0)

    ax[1,ireg].set_xlabel('Target (month)', fontsize=labelfontsize)
    ax[1,ireg].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[1,ireg].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[1,ireg].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[1,ireg].set_xticklabels(month_letters, fontsize=labelfontsize)
#     ax[1,ireg].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)

    if ireg == 0:
        ax[1,ireg].set_ylabel('Lead (month)', fontsize=labelfontsize)
        ax[1,ireg].text(-5.3, -maxlag/2+0.75, 'correlation (NPP, SFC_IRR)', clip_on=False, rotation='vertical', ha='center', va='center', fontsize=titlefontsize, fontweight='bold')
        
# fig.suptitle(r'correlation ( '+var_su_names[predictand]+'$_{target\:month}$, '+var_su_names[predictor]+'$_{lead\:month}$ )', ha='center', x=0.45, fontweight='bold', fontsize=titlefontsize+4)
# fig.tight_layout()

cb = fig.colorbar(im, ax=ax.ravel().tolist(), ticks=[-1,-0.5,0,0.5,1], shrink=0.6, pad=0.02)
cb.set_label('Correlation coefficient', fontsize=labelfontsize)
cb.ax.tick_params(labelsize=labelfontsize)

SIE
SouthernOcean
(14, 12)
[[-0.6  -0.38 -0.47 -0.66 -0.74 -0.6  -0.54 -0.54 -0.59 -0.62 -0.77 -0.56]
 [-0.29 -0.44 -0.44 -0.34 -0.63 -0.58 -0.57 -0.61 -0.6  -0.56 -0.71 -0.41]
 [-0.24 -0.36 -0.48 -0.3  -0.27 -0.48 -0.51 -0.58 -0.61 -0.53 -0.69 -0.5 ]
 [-0.33 -0.33 -0.41 -0.34 -0.26 -0.23 -0.39 -0.5  -0.56 -0.5  -0.62 -0.51]
 [-0.33 -0.39 -0.34 -0.38 -0.24 -0.21 -0.17 -0.39 -0.5  -0.42 -0.52 -0.45]
 [-0.28 -0.39 -0.34 -0.33 -0.35 -0.17 -0.17 -0.21 -0.43 -0.37 -0.39 -0.37]
 [-0.2  -0.33 -0.35 -0.28 -0.28 -0.28 -0.11 -0.21 -0.23 -0.37 -0.3  -0.29]
 [-0.14 -0.23 -0.28 -0.27 -0.18 -0.16 -0.28 -0.24 -0.21 -0.22 -0.32 -0.22]
 [-0.09 -0.16 -0.2  -0.23 -0.15 -0.08 -0.17 -0.39 -0.27 -0.19 -0.29 -0.22]
 [-0.11 -0.12 -0.14 -0.18 -0.11 -0.06 -0.12 -0.3  -0.49 -0.3  -0.23 -0.2 ]
 [-0.09 -0.15 -0.1  -0.14 -0.09 -0.03 -0.1  -0.26 -0.5  -0.45 -0.33 -0.14]
 [-0.08 -0.18 -0.13 -0.13 -0.07 -0.03 -0.07 -0.25 -0.47 -0.51 -0.53 -0.2 ]
 [-0.08 -0.17 -0.16 -0.15 -0.07 -0.03 -0.07 -0.23 -0.45 -0.51 -0.64 -0.39

<Figure size 6500x3000 with 13 Axes>

### LIEBIG_LIM_SM predicting NPP and JPROD_NSM_100

In [24]:
# predictor
# predictand
maxlag=-13
limsm_pred_npp_rthresh = np.array([0.20836084, 0.22937294, 0.24829483, 0.24642464, 0.16336634, 0.16622662])
limsm_pred_jprodnsm_rthresh = np.array([0.20836084, 0.20275028, 0.24268427, 0.23740374, 0.16160616, 0.15313531])
hatch='...'
figsize=(13,6)
labelfontsize=10
titlefontsize=12


plt.rcParams.update({'mathtext.default':'regular'})

reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names = get_plotting_labels()
region_list = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

fig,ax = plt.subplots(2, 6, figsize=figsize, constrained_layout=True)

subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('liebig_lim_sm', 'so', 'anom', ctrl_type='additional_')[reg]
    ts_predictand = open_metric('jprod_ntot_100', 'so', 'anom', ctrl_type='additional_')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed
    
    print('LIEBIG_LIM_SM pred. NPP')
    print(reg)
    print(r)

    im = ax[0,ireg].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > limsm_pred_npp_rthresh[ireg], 0, 1))
    masked_im = ax[0,ireg].pcolor(masked, hatch=hatch, alpha=0)

    ax[0,ireg].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[0,ireg].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[0,ireg].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[0,ireg].set_xticklabels(month_letters, fontsize=labelfontsize)
    ax[0,ireg].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)

    if ireg == 0:
        ax[0,ireg].set_ylabel('Lead (month)', fontsize=labelfontsize)
        ax[0,ireg].text(-5.3, -maxlag/2+0.75, 'correlation\n(NPP, LIEBIG_LIM_SM)',
                        clip_on=False, rotation='vertical', ha='center', va='center',
                        fontsize=titlefontsize-3, fontweight='bold')


for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('liebig_lim_sm', 'so', 'anom', ctrl_type='additional_')[reg]
    ts_predictand = open_metric('jprod_nsm_100', 'so', 'anom', ctrl_type='additional_')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed
    
    print('LIEBIG_LIM_SM pred. JPROD_NSM_100')
    print(reg)
    print(r)

    im = ax[1,ireg].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > limsm_pred_jprodnsm_rthresh[ireg], 0, 1))
    masked_im = ax[1,ireg].pcolor(masked, hatch=hatch, alpha=0)

    ax[1,ireg].set_xlabel('Target (month)', fontsize=labelfontsize)
    ax[1,ireg].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[1,ireg].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[1,ireg].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[1,ireg].set_xticklabels(month_letters, fontsize=labelfontsize)

    if ireg == 0:
        ax[1,ireg].set_ylabel('Lead (month)', fontsize=labelfontsize)
        ax[1,ireg].text(-5.3, -maxlag/2+0.75, 'correlation\n(JPROD_NSM_100, LIEBIG_LIM_SM)',
                        clip_on=False, rotation='vertical', ha='center', va='center',
                        fontsize=titlefontsize-3, fontweight='bold')
        
# fig.suptitle(r'correlation ( '+var_su_names[predictand]+'$_{target\:month}$, '+var_su_names[predictor]+'$_{lead\:month}$ )', ha='center', x=0.45, fontweight='bold', fontsize=titlefontsize+4)
# fig.tight_layout()

cb = fig.colorbar(im, ax=ax.ravel().tolist(), ticks=[-1,-0.5,0,0.5,1], shrink=0.6, pad=0.02)
cb.set_label('Correlation coefficient', fontsize=labelfontsize)
cb.ax.tick_params(labelsize=labelfontsize)

FileNotFoundError: [Errno 2] No such file or directory: b'/local/projects/so_predict/bbuchovecky/storage/so_predict_additional_derived/CTRL/LIEBIG_LIM_SM/liebig_lim_sm_ts_so_monthly_anom.nc'

<Figure size 6500x3000 with 12 Axes>

## Weddell MLD and NPP PPP

In [26]:
mld_ctl = xr.open_dataset('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/CTRL/MLD/mld_ts_so_annual_mean.nc')
mld_ens_annual = xr.open_dataset('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/MLD_ENSEMBLE/mld_ens_so_annual_mean.nc')
mld_ens_month = xr.open_dataset('/local/projects/so_predict/bbuchovecky/storage/so_predict_derived/MLD_ENSEMBLE/mld_ens_so_monthly_mean.nc')

In [27]:
sorted(mld_ens_annual.nStart)

[<xarray.DataArray 'nStart' ()>
 array('0022_0031', dtype='<U9')
 Coordinates:
     nStart   <U9 '0022_0031',
 <xarray.DataArray 'nStart' ()>
 array('0064_0073', dtype='<U9')
 Coordinates:
     nStart   <U9 '0064_0073',
 <xarray.DataArray 'nStart' ()>
 array('0106_0115', dtype='<U9')
 Coordinates:
     nStart   <U9 '0106_0115',
 <xarray.DataArray 'nStart' ()>
 array('0170_0179', dtype='<U9')
 Coordinates:
     nStart   <U9 '0170_0179',
 <xarray.DataArray 'nStart' ()>
 array('0232_0241', dtype='<U9')
 Coordinates:
     nStart   <U9 '0232_0241',
 <xarray.DataArray 'nStart' ()>
 array('0295_0304', dtype='<U9')
 Coordinates:
     nStart   <U9 '0295_0304']

In [28]:
mld_ens_annual['Weddell'].sel(nStart=s,nEns=n).values.max()

NameError: name 's' is not defined

In [29]:
fig,ax = plt.subplots(figsize=(8,4))
ax.plot(np.arange(1,301), mld_ctl['Weddell'], c='k', lw=1)
ax.set_xlim(0,305)

for i,s in enumerate(sorted(mld_ens_annual['nStart'].values)):
    nStart = int(s.split('_')[0])
    
    mx = 0
    for n in mld_ens_annual['nEns']:
        ax.plot(np.arange(10)+nStart, mld_ens_annual['Weddell'].sel(nStart=s,nEns=n), c='r', lw=0.5)
        if mld_ens_annual['Weddell'].sel(nStart=s,nEns=n).max() > mx:
            mx = mld_ens_annual['Weddell'].sel(nStart=s,nEns=n).max()
        
    ax.text(nStart, mx+5, 'ENS0%d'%(i+1), ha='center', fontsize=8)
    
ax.set_ylabel('MLD [m]')
ax.set_xlabel('Time [year]')


Text(0.5, 0, 'Time [year]')

<Figure size 4000x2000 with 1 Axes>

In [30]:
sorted(ppp.nStart.values)

NameError: name 'ppp' is not defined

In [33]:
ppp = open_metric('npp','so', 'ppp', ens_type='indv_ens')['Weddell']

fig,axes = plt.subplots(2,3,figsize=(12,5))
ax = axes.ravel()

title = ['a', 'b', 'c', 'd', 'e', 'f']
ens_titles = ['ENS01', 'ENS02', 'ENS03', 'ENS04', 'ENS05', 'ENS06']
ens_start_yrs = [170, 22, 64, 106, 232, 295]
for i,st in enumerate(sorted(ppp.nStart.values)):
    years = str(int(st.split('_')[0]))+'-'+str(int(st.split('_')[1]))
    ax[i].plot(np.arange(1,121), ppp.sel(nStart=st), c='lightgreen', lw=2)
    
    ax[i].set_xlim(1,120)
    ax[i].set_xticks(np.arange(0,121,24))
    ax[i].set_xticklabels(np.arange(0,11,2))
    ax[i].set_ylim(-0.5,1)
    ax[i].set_yticks(np.arange(-0.4,1.1,0.2))
    ax[i].hlines(0,0,121,lw=0.75,color='k')
    
#     ax[i].set_title('(%s) %s, years %d-%d' % (title[i], ens_titles[i], ens_start_yrs[i], ens_start_yrs[i]+10),
#                     loc='left', fontweight='bold')
    ax[i].set_title('%s (years %s)' % (ens_titles[i],years),
                    loc='left')
    
for a in ax[3:]:
    a.set_xlabel('Lead time [year]')
for a in ax[::3]:
    a.set_ylabel('PPP')

fig.tight_layout()

<Figure size 6000x2500 with 6 Axes>

In [35]:
fig = plt.figure(figsize=(12,9))

gs = fig.add_gridspec(3, 3,  width_ratios=(1,1,1), height_ratios=(2,1,1),
#                       left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.2, hspace=0.35
                     )

axmld = fig.add_subplot(gs[0, :2])

ax = []
for i in range(6):
    a = fig.add_subplot(gs[i//3+1,i%3])
    ax.append(a)
    
## MLD
axmld.plot(np.arange(1,301), mld_ctl['Weddell'], c='k', lw=1, label='Control simulation')
axmld.set_xlim(0,305)

for ixs,s in enumerate(sorted(mld_ens_annual['nStart'].values)):
    nStart = int(s.split('_')[0])
    
    mx = 0
    for ixn,n in enumerate(mld_ens_annual['nEns']):
        if ixs == 0 and ixn == 0:
            axmld.plot(np.arange(10)+nStart, mld_ens_annual['Weddell'].sel(nStart=s,nEns=n), c='r', lw=0.5, label='Ensemble members')
        else:
            axmld.plot(np.arange(10)+nStart, mld_ens_annual['Weddell'].sel(nStart=s,nEns=n), c='r', lw=0.5)
        
        if mld_ens_annual['Weddell'].sel(nStart=s,nEns=n).max() > mx:
            mx = mld_ens_annual['Weddell'].sel(nStart=s,nEns=n).max()
        
    axmld.text(nStart, mx+5, 'ENS0%d'%(ixs+1), ha='center', fontsize=8)
    
axmld.set_ylabel('MLD [m]')
axmld.set_xlabel('Time [year]')

leg = axmld.legend(ncol=1, bbox_to_anchor=[1,1])
for line in leg.get_lines():
    line.set_linewidth(3.0)


## PPP
ppp = open_metric('npp','so', 'ppp', ens_type='indv_ens')['Weddell']

title = ['a', 'b', 'c', 'd', 'e', 'f']
ens_titles = ['ENS01', 'ENS02', 'ENS03', 'ENS04', 'ENS05', 'ENS06']
ens_start_yrs = [170, 22, 64, 106, 232, 295]
for i,st in enumerate(sorted(ppp.nStart.values)):
    years = str(int(st.split('_')[0]))+'-'+str(int(st.split('_')[1]))
    ax[i].plot(np.arange(1,121), ppp.sel(nStart=st), c='lightgreen', lw=2)
    
    ax[i].set_xlim(1,120)
    ax[i].set_xticks(np.arange(0,121,24))
    ax[i].set_xticklabels(np.arange(0,11,2))
    ax[i].set_ylim(-0.5,1)
    ax[i].set_yticks(np.arange(-0.4,1.1,0.2))
    ax[i].hlines(0,0,121,lw=0.75,color='k')
    
#     ax[i].set_title('(%s) %s, years %d-%d' % (title[i], ens_titles[i], ens_start_yrs[i], ens_start_yrs[i]+10),
#                     loc='left', fontweight='bold')
    ax[i].set_title('%s (years %s)' % (ens_titles[i],years),
                    loc='left')
    
for a in ax[3:]:
    a.set_xlabel('Lead time [year]')
for a in ax[::3]:
    a.set_ylabel('PPP')

<Figure size 6000x4500 with 7 Axes>

---

In [43]:
predictor
predictand
maxlag=-13
sie_pred_npp_rthresh = np.array([0.28028028, 0.15115115, 0.18318318, 0.24524525, 0.15115115, 0.14614615])
irr_pred_npp_rthresh = np.array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915, 0.12912913])
hatch='...'
figsize=(6,12)
labelfontsize=8
titlefontsize=10


plt.rcParams.update({'mathtext.default':'regular'})

reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names = get_plotting_labels()
region_list = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

fig,ax = plt.subplots(6, 2, figsize=figsize)

subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('SIE', 'so', 'anom')[reg]
    ts_predictand = open_metric('NPP', 'so', 'anom')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed

    im = ax[ireg,0].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > sie_pred_npp_rthresh[ireg], 0, 1))
    masked_im = ax[ireg,0].pcolor(masked, hatch=hatch, alpha=0)

    ax[ireg,0].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[ireg,0].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[ireg,0].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[ireg,0].set_xticklabels(month_letters, fontsize=labelfontsize)
    ax[ireg,0].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)

    ax[ireg,0].set_ylabel('Lead (month)', fontsize=labelfontsize)
    
    if ireg == 5:
        ax[ireg,0].set_xlabel('Target (month)', fontsize=labelfontsize)

for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric('SFC_IRR', 'so', 'anom')[reg]
    ts_predictand = open_metric('NPP', 'so', 'anom')[reg]

    r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)

    ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
    numlags = r.shape[0]
    r_transformed = np.empty(shape=r.shape)
    p_transformed = np.empty(shape=p.shape)
    for i in range(numlags):
        r_transformed[numlags-i-1] = r[i]
        p_transformed[numlags-i-1] = p[i]
    r = r_transformed
    p = p_transformed

    im = ax[ireg,1].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='gray', linewidth=0.01)

    masked = np.ma.masked_array(r, mask=np.where(abs(r) > irr_pred_npp_rthresh[ireg], 0, 1))
    masked_im = ax[ireg,1].pcolor(masked, hatch=hatch, alpha=0)

    ax[ireg,1].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
    ax[ireg,1].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize=labelfontsize)
    ax[ireg,1].set_xticks((np.arange(r.shape[1]) + 0.5))
    ax[ireg,1].set_xticklabels(month_letters, fontsize=labelfontsize)
#     ax[ireg,1].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left', fontsize=titlefontsize)

    if ireg == 5:
        ax[ireg,1].set_xlabel('Target (month)', fontsize=labelfontsize)
        
        
# fig.suptitle(r'correlation ( '+var_su_names[predictand]+'$_{target\:month}$, '+var_su_names[predictor]+'$_{lead\:month}$ )', ha='center', x=0.45, fontweight='bold', fontsize=titlefontsize+4)
fig.tight_layout()

# cb = fig.colorbar(im, ax=ax.ravel().tolist())
# cb.set_label('Correlation coefficient', fontsize=labelfontsize)

<Figure size 1800x3600 with 12 Axes>